In [331]:
# import pandas as pd
# import math
# import matplotlib.pyplot as plt
import numpy as np

import gurobipy as gp
from gurobipy import GRB, tuplelist

# import pandapower.networks as pn
# import scipy.io

In [332]:
# m = gp.Model('LinMcCormick')
idx_x = np.arange(1,9,1)
set_x = tuplelist([i for i in idx_x])

In [333]:
def define_variables(m):
    # X
    x = m.addVars(set_x, name='x') 
    x[1].LB, x[1].UB = 100, 10000

    for i in [2,3]:
        x[i].LB, x[i].UB = 1000, 10000

    for i in [4,5,6,7,8]:
        x[i].LB, x[i].UB = 10, 1000

    return x

In [334]:
def define_objective(m,x):
    sum_x = sum(x[i] for i in [1,2,3])

    m.setObjective(sum_x, GRB.MINIMIZE)

In [335]:
def nonlinear_constraints(m,x):
    m.addConstr(0.0025*(x[4] + x[6]) - 1 <= 0)
    m.addConstr(0.0025*(-x[4] + x[5] + x[7]) - 1 <= 0)
    m.addConstr(0.01*(-x[5] + x[8]) - 1 <= 0)
    m.addConstr(100*x[1] - x[1]*x[6] + 833.33252*x[4] - 83333.333 <= 0)
    m.addConstr(x[2]*x[4] - x[2]*x[7] - 1250*x[4] + 1250*x[5] <= 0)
    m.addConstr(x[3]*x[5] - x[3]*x[8] - 2500*x[5] + 1250000 <= 0)

In [336]:
def mc_cormick(m,a,b,nm):
    c = m.addVar(name=nm)
    m.addConstr(c >= a.LB*b + a*b.LB - a.LB*b.LB)
    m.addConstr(c >= a.UB*b + a*b.UB - a.UB*b.UB)
    m.addConstr(c <= a.LB*b + a*b.UB - a.LB*b.UB)
    m.addConstr(c <= a.UB*b + a*b.LB - a.UB*b.LB)

    return c

In [337]:
def linear_constraint(m,x,c16,c24,c27,c35,c38):
    m.addConstr(0.0025*(x[4] + x[6]) - 1 <= 0)
    m.addConstr(0.0025*(-x[4] + x[5] + x[7]) - 1 <= 0)
    m.addConstr(0.01*(-x[5] + x[8]) - 1 <= 0)
    m.addConstr(100*x[1] - c16 + 833.33252*x[4] - 83333.333 <= 0)
    m.addConstr(c24 - c27 - 1250*x[4] + 1250*x[5] <= 0)
    m.addConstr(c35 - c38 - 2500*x[5] + 1250000 <= 0)

In [338]:
def var_values(y,mult=1):
    z = []
    for v in y.values():
        z.append(v.X*mult)
    return z

In [339]:
## Nonlinear - nonconvex
m = gp.Model('NonLinMcCormick')
x = define_variables(m)
m.update()
define_objective(m,x)
nonlinear_constraints(m,x)

# m.write('nonlinear_mc_cormick.lp')
m.Params.OutputFlag = 0
# m.Params.SolutionLimit = 1
m.optimize()

print(m.getObjective().getValue())
print(var_values(x))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 8 columns and 7 nonzeros
Model fingerprint: 0x7c3dd7c0
Model has 3 quadratic constraints
Coefficient statistics:
  Matrix range     [3e-03, 1e-02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+02, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+04]
  RHS range        [1e+00, 1e+00]
  QRHS range       [8e+04, 1e+06]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 15 rows, 12 columns, 44 nonzeros
Presolved model has 5 bilinear constraint(s)
Variable types: 12 continuous, 0 integer (0 binary)
Found heuristic solution: objective 7049.2480205

Root relaxation: objective 2.717132e+03, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Cu

In [340]:
## Relaxation - basic McCormick linearization
n = gp.Model('LinMcCormick')
y = define_variables(n)
n.update()
c16 = mc_cormick(n,y[1],y[6],'c16')
c24 = mc_cormick(n,y[2],y[4],'c24')
c27 = mc_cormick(n,y[2],y[7],'c27')
c35 = mc_cormick(n,y[3],y[5],'c35')
c38 = mc_cormick(n,y[3],y[8],'c38')

define_objective(n,y)
linear_constraint(n,y,c16,c24,c27,c35,c38)

n.write('linear_mc_cormick.lp')
n.Params.OutputFlag = 0
m.Params.NonConvex = 2
# m.Params.SolutionLimit = 1
n.optimize()


print(n.getObjective().getValue())
print(var_values(y))

Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 26 rows, 13 columns and 77 nonzeros
Model fingerprint: 0x2e1f18af
Coefficient statistics:
  Matrix range     [3e-03, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+04]
  RHS range        [1e+00, 1e+07]
Presolve time: 0.01s
Presolved: 26 rows, 13 columns, 77 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1000000e+03   2.045313e+04   0.000000e+00      0s
      14    2.5332008e+03   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.533200803e+03
2533.200802906578
[352.34869495741, 1180.852107949168, 1000.0, 364.765130504259, 460.0, 35.234869495740995, 304.765130

In [342]:
print('Runtime difference =', m.Runtime - n.Runtime)
print('Objective difference =', m.getObjective().getValue() - n.getObjective().getValue())
print('Objective_m = ', m.getObjective().getValue())
x

Runtime difference = 0.13900017738342285
Objective difference = 4516.047217631989
Objective_m =  7049.248020538567


{1: <gurobi.Var x[1] (value 579.3066844257957)>,
 2: <gurobi.Var x[2] (value 1359.9706680592556)>,
 3: <gurobi.Var x[3] (value 5109.9706680535155)>,
 4: <gurobi.Var x[4] (value 182.01769958088101)>,
 5: <gurobi.Var x[5] (value 295.60117327799134)>,
 6: <gurobi.Var x[6] (value 217.98230041878287)>,
 7: <gurobi.Var x[7] (value 286.4165263027629)>,
 8: <gurobi.Var x[8] (value 395.60117327795905)>}